In [6]:
import numpy as np
A=np.array([[11,12,13],[21,22,23],[31,32,33]])
B=np.ones(shape=(3,3))

In [1]:
import tensorflow as tf


#### 基本运算；常量、变量、占位符

In [8]:
a=tf.constant([1,2],name="a")
b=tf.constant([2,4],name="b")
result = a+b
print(result)

#上面只是定义了计算图，并没有运行计算图，所以不会输出运算结果

sess=tf.Session()
a=sess.run(result)
print(a)
sess.close

#打开会话，运行计算图，关闭计算图

Tensor("add:0", shape=(2,), dtype=int32)
[3 6]


<bound method BaseSession.close of <tensorflow.python.client.session.Session object at 0x000001F5A9E4E4E0>>

In [9]:
with tf.Session() as sess:
    a=tf.constant([1,2,3,4])
    b=tf.constant([1,2,3,4])
    result=tf.multiply(a,b)
    c=sess.run(result)
    print(c)
    
print(c)

# with 结束，计算会话自动关闭

[ 1  4  9 16]
[ 1  4  9 16]


In [10]:
import tensorflow as tf
#（2，3，1单元）3层前向神经网络（无激活函数）

#生成服从标准差为1的正态分布的随机变量，作为初始化矩阵
w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

x=tf.constant([[0.7,0.9]])
#矩阵乘法
a=tf.matmul(x,w1)
b=tf.matmul(a,w2)

sess=tf.Session()
#需要运行初始化赋值，前面只是定义，没运算
sess.run(w1.initializer)
sess.run(w2.initializer)
y=sess.run(b)

sess.close()
print(y)

with tf.Session() as sess:
    w3=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
    w4=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))
    x1=tf.constant([[0.7,0.9]])
    a1=tf.matmul(x,w3)
    b1=tf.matmul(a1,w4)
    #可以直接嵌入初始化
    sess.run(tf.global_variables_initializer())
    print(sess.run(b1))


[[ 3.95757794]]
[[ 3.95757794]]


In [11]:
import tensorflow as tf
w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

#因为需要重复输入x，而每建一个x就会生成一个结点，计算图的效率会低。所以使用占位符
x=tf.placeholder(tf.float32,shape=(1,2))
x1=tf.placeholder(tf.float32,shape=(3,2))
a=tf.matmul(x,w1)
a1=tf.matmul(x1,w1)
y=tf.matmul(a,w2)
y1=tf.matmul(a1,w2)

sess=tf.Session()
sess.run(tf.global_variables_initializer())
#运行y时将占位符填上，feed_dict为字典，变量名不可变
y_hat=sess.run(y,feed_dict={x:[[0.7,0.9]]})
y_hat1=sess.run(y1,feed_dict={x1:[[0.7,0.9],[0.2,0.5],[1,2]]})# batch = 3
print(y_hat)
print(y_hat1)
sess.close


[[ 3.95757794]]
[[ 3.95757794]
 [ 1.657197  ]
 [ 7.20209646]]


<bound method BaseSession.close of <tensorflow.python.client.session.Session object at 0x000001F5A9F53F60>>

#### 简单的分类模型

In [ ]:
import tensorflow as tf
from numpy.random import RandomState

batch_size=10
w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

# None 可以根据batch 大小确定维度，在shape的一个维度上使用None，方便不大的batch
x=tf.placeholder(tf.float32,shape=(None,2))
y=tf.placeholder(tf.float32,shape=(None,1))

a=tf.matmul(x,w1)
yhat=tf.matmul(a,w2)

#定义交叉熵为损失函数，训练过程使用Adam算法最小化交叉熵
cross_entropy=-tf.reduce_mean(y*tf.log(tf.clip_by_value(yhat,1e-10,1.0)))
train_step=tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

rdm=RandomState(1)
data_size=516

#生成两个特征，共data_size个样本
X=rdm.rand(data_size,2)
#定义规则给出样本标签，所有x1+x2<1的样本认为是正样本，其他为负样本。Y，1为正样本
Y = [[int(x1+x2 < 1)] for (x1, x2) in X]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(w1))
    print(sess.run(w2))
    steps=11000
    for i in range(steps):
        
        #选定每一个批量读取的首尾位置，确保在1个epoch内采样训练
        start = i * batch_size % data_size
        end = min(start + batch_size,data_size)
        sess.run(train_step,feed_dict={x:X[start:end],y:Y[start:end]})
        if i % 1000 == 0:
            training_loss= sess.run(cross_entropy,feed_dict={x:X,y:Y})
            print("在迭代 %d 次后，训练损失为 %g"%(i,training_loss))

#### 激活函数和偏置项：
a=tf.nn.relu(tf.matmul(x,w1)+biases1)

yhat=tf.nn.relu(tf.matmul(a,w2)+biases2)

#### 交叉熵函数
cross_entropy=-tf.reduce_mean(y*tf.log(tf.clip_by_value(yhat,1e-10,1.0)))

tf.reduce_mean(x)表示计算全局平均值。tf.clip_by_value()函数可以将张量中的数值限制在一个范围内。tf.log()对张量内的所有元素依次求对数。交叉熵函数一般会与softmax回归一起使用，TensorFlow将它们进行了统一封装：cross_entropy=tf.nn.softmax_cross_entropy_with_logits(yhat,y)



#### 均方误差损失函数
mse=tf.reduce_mean(tf.square(y-yhat))

tf.select(tf.greater(y,yhat),y-yhat,yhat-y)

tf.greater()的输入是两个张量，比较两个张量中的每一个元素，并返回比较结果（true或false的向量）。tf.select()有三个参数，第一个参数条件为真时选择第二个参数中的值，否则选择第三个参数的值。

In [10]:
#decayed_learning_rate=learning_rate*decay_rate^(global_steps/decay_steps)，指数衰减函数的定义

global_step=tf.Variable(0)
#使用exponential_decay生成学习速率，因为staircase=tire，每100次迭代，学习率×0.96
learning_rate=tf.train.exponential_decay(0.1,global_step,100,0.96,staircase=True)
#在minimize中导入global_step将自动更新
#learning_step=tf.train.GtadientDescentOptimizer(learning_rate).minimize(loss_function,global_step=global_step)


#### 带L2正则化的损失函数
w=tf.Variable(tf.random_normal([2,1],stddev=1,seed=1))

yhat=tf.matmul(x,w)

loss=tf.reduce_mean(tf.square(y-yhat))+tf.contrib.layers.l2_regularizer(lambda)(w)

In [1]:
import tensorflow as tf
w=tf.constant([[1.0,-5.0],[-3.0,4.0]])
with tf.Session() as sess:
    #L1正则化：(1+5+3+4)×0.5
    print(sess.run(tf.contrib.layers.l1_regularizer(0.5)(w)))
    #L2正则化：（1+25+9+16）/2×0.5，L2正则化会处以2，无偏估计？
    print(sess.run(tf.contrib.layers.l2_regularizer(0.5)(w)))

6.5
12.75


神经网络结构复杂后，定义网络结构的部分和计算损失函数的部分可能不在同一个函数中。所以采用collection在一个计算图中保留一组实体（如张量）。

In [2]:
import tensorflow as tf

#通过集合（collection）计算一个5层神经网络带L2正则化的损失函数

#随机正态初始化一层神经网络的权重，并将权重的L2正则化损失加入名为losses的集合中，返回初始化的权重
def get_weight(shape,lambd):
    var=tf.Variable(tf.random_normal(shape),dtype=tf.float32)
    
    #tf.add_to_collection函数将新生成变量的L2正则化损失项加入集合，第一个参数为集合名，第二个参数为加入集合的内容。
    tf.add_to_collection('losses',tf.contrib.layers.l2_regularizer(lambd)(var))
    return var

x=tf.placeholder(tf.float32,shape=(None,2))
y=tf.placeholder(tf.float32,shape=(None,1))
batch_size=8

#定义每一层中结点个数和层数
layer_dimension=[2,10,10,10,1]
n_layers=len(layer_dimension)

#该变量维护前向传播时最深层的结点，最开始为输入层
cur_layer=x

#输入层结点个数
in_dimension=layer_dimension[0]

#通过循环生成5层全连接神经网络
for i in range(1,n_layers):
    
    #下一层节点数
    out_dimension=layer_dimension[i]
    
    #生成当前层中权重的变量，并将这个变量的L2正则化损失加入计算图上的集合
    #[in_dimension,out_dimension],例第一层到第二层之间的权重维度为2×10
    weight=get_weight([in_dimension,out_dimension],0.001)
    
    #偏置项和后一层维度相等，为什么是wx+0.1，而不是wx+b？？
    bias=tf.Variable(tf.constant(0.1,shape=[out_dimension]))
    
    #使用ReLU激活函数，cur_layer储存传播一层后的激活情况，后一层激活函数的输出
    cur_layer=tf.nn.relu(tf.matmul(cur_layer,weight)+bias)
    in_dimension=layer_dimension[i]
    
mse_loss=tf.reduce_mean(tf.square(y-cur_layer))

#将均方误差函数加入损失集合
tf.add_to_collection('losses',mse_loss)

#get_collection返回一个列表，这个列表是集合中的所有元素，这些元素就是组成损失函数的误差和正则项，相加得最终损失函数
loss=tf.add_n(tf.get_collection('losses'))



In [4]:
import tensorflow as tf

#tf.assign(A, new_number),这个函数的功能主要是把A的值变为new_number
A=tf.Variable(tf.constant(0.0),dtype=tf.float32)
with tf.Session() as sess:  
    sess.run(tf.global_variables_initializer())  
    print (sess.run(A))  
    sess.run(tf.assign(A, 10))  
    print (sess.run(A))  

0.0
10.0


In [2]:
import tensorflow as tf

#滑动平均模型

#定义一个变量计算滑动平均，初始值为0，所有需要计算滑动平均的变量必须是实数型
v1=tf.Variable(0,dtype=tf.float32)

#step变量模拟神经网络中的迭代次数，用于动态控制衰减率
step=tf.Variable(0,trainable=False)

#定义一个滑动平均的类，初始化时给定了衰减率和控制衰减率的变量step
ema=tf.train.ExponentialMovingAverage(0.99,step)

#定义一个更新变量的滑动平均操作，给定一个列表，每次执行操作时更新列表所有变量
maintain_averages_op=ema.apply([v1])

with tf.Session() as sess:
    
    #初始化所有变量
    sess.run(tf.global_variables_initializer())
    
    #通过ema.average(v1)获取滑动平均之后变量的取值。初始化后，v1的值和滑动平均都为0
    print(sess.run([v1,ema.average(v1)]))
    
    #更新变量v1的值为5,tf.assign将数值分配给变量
    sess.run(tf.assign(v1,5))
    
    #更新v1的滑动平均值。衰减率为min{0.99,(1+step)/(10+step)≈0.1}=0.1，所以v1的滑动平均值会更新为 0.1×0+0.9×5=4.5
    sess.run(maintain_averages_op)
    print(sess.run([v1,ema.average(v1)]))
    
    #将迭代设置为10000步
    sess.run(tf.assign(step,10000))
    
    #更新v1的值为10
    sess.run(tf.assign(v1,10))
    
    #更新v1的滑动平均值。衰减率为min{0.99,(1+step)/(10+step)≈0.999}=0.99，所以v1的滑动平均会被更新为0.99×4.5+0.01×10=4.555
    sess.run(maintain_averages_op)
    print(sess.run([v1,ema.average(v1)]))
    
    #再次更新滑动平均值，得到新的滑动平均值为0.99×4.555+0.01×10=4.60945
    sess.run(maintain_averages_op)
    print(sess.run([v1,ema.average(v1)]))



[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.5549998]
[10.0, 4.6094499]
